In [2]:
# #colabを使う方はこちらを使用ください。
# !pip install torch==0.4.1
# !pip install torchvision==0.2.1


    100% |████████████████████████████████| 61kB 2.3MB/s 
    100% |████████████████████████████████| 519.5MB 37kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x59776000 @  0x7f62790a81c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 2.0MB 5.4MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0
    100% |████████████████████████████████| 5.8MB 926kB/s 
  Found existing installation: Pillow 5.2.0
    Uninstalling Pillow-5.2.0:
      Successfully uninstalled Pillow-5.2.0


In [6]:
import torch
torch.__version__

'0.4.1'

In [0]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt

# ネットワーク定義

### Sequentialを使った書き方



In [101]:
model = nn.Sequential(
          nn.Conv2d(1,20,5),
          nn.ReLU(),
          nn.Conv2d(20,64,5),
          nn.ReLU()
        )
model

Sequential(
  (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
  (3): ReLU()
)

In [102]:
model = torch.nn.Sequential()
model.add_module("conv1", nn.Conv2d(1,20,5))
model.add_module("relu1", nn.ReLU())
model.add_module("conv2", nn.Conv2d(20,64,5))
model.add_module("relu2", nn.ReLU())
model

Sequential(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (conv2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU()
)

In [104]:
from collections import OrderedDict
model = nn.Sequential(OrderedDict([
          ('conv1', nn.Conv2d(1,20,5)),
          ('relu1', nn.ReLU()),
          ('conv2', nn.Conv2d(20,64,5)),
          ('relu2', nn.ReLU())
        ]))
model

Sequential(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (conv2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU()
)

## nn.Moduleを継承した書き方


In [5]:
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)
        self.conv2 = nn.Conv2d(20, 20, 5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        return F.relu(self.conv2(x))
      
model = Model()
model

Model(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 20, kernel_size=(5, 5), stride=(1, 1))
)

## nn.ModuleListを使った書き方


In [8]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.convs = nn.ModuleList([nn.Conv2d(1, 20, 5), nn.Conv2d(20, 20, 5)])

    def forward(self, x):
        for i, l in enumerate(self.convs):
            x = l(x)
        return x
      
model = Model()
model

Model(
  (convs): ModuleList(
    (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
    (1): Conv2d(20, 20, kernel_size=(5, 5), stride=(1, 1))
  )
)

## nn.ModuleDictを使った書き方

In [11]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.convs = nn.ModuleDict({'conv1' : nn.Conv2d(1, 20, 5), 'conv2' : nn.Conv2d(20, 20, 5)})

    def forward(self, x):
        for l in self.convs.values():
            x = l(x)
        return x
      
model = Model()
model

Model(
  (convs): ModuleDict(
    (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
    (conv2): Conv2d(20, 20, kernel_size=(5, 5), stride=(1, 1))
  )
)

## 自動微分機能

In [4]:
# テンソルを作る
x = torch.tensor(1, requires_grad=False, dtype=torch.float32)
w = torch.tensor(2, requires_grad=True, dtype=torch.float32)
b = torch.tensor(3, requires_grad=True, dtype=torch.float32)

# 計算グラフを作成する
y = w * x + b    # y = 1 * 2 + 3

# テンソルを返す
print("y：", y)

# 勾配計算
y.backward()

# 勾配を確認
print("x.grad：", x.grad) #requires_grad=Falseなので微分計算がない。
print("w.grad：", w.grad)
print("b.grad：", b.grad)

y： tensor(5., grad_fn=<ThAddBackward>)
x.grad： None
w.grad： tensor(1.)
b.grad： tensor(1.)


In [5]:
#ネットワークのパラメータを凍結する
net = torchvision.models.vgg16(pretrained=True)
for param in net.features.parameters():
    param.requires_grad = False

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.torch/models/vgg16-397923af.pth
100%|██████████| 553433881/553433881 [00:19<00:00, 27764332.76it/s]


## GPUを使用する

- PyTorch 0.4.0以前

In [6]:
xf_gpu = torch.FloatTensor(1, 2).cuda()
xf_gpu

tensor([[0.0000, 0.0000]], device='cuda:0')

In [7]:
xf_cpu = xf_gpu.cpu()
xf_cpu

tensor([[0.0000, 0.0000]])

In [8]:
xf_numpy = xf_cpu.numpy()
xf_numpy

array([[9.473915e-37, 0.000000e+00]], dtype=float32)

- PyTorch 0.4.0, PyTorch 0.4.1

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
xf_torch = torch.FloatTensor(1, 2).to(device)
xf_torch

tensor([[1.2048e-02, 4.5787e-41]], device='cuda:0')

In [10]:
xf_cpu = xf_gpu.to('cpu')
xf_cpu

tensor([[0.0000, 0.0000]])

- PyTorch 0.4.0以前

In [11]:
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)
        self.conv2 = nn.Conv2d(20, 20, 5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        return F.relu(self.conv2(x))
net = Model()
net.cuda()

Model(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 20, kernel_size=(5, 5), stride=(1, 1))
)

- PyTorch 0.4.0, PyTorch 0.4.1

In [12]:
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)
        self.conv2 = nn.Conv2d(20, 20, 5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        return F.relu(self.conv2(x))
net = Model()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net.to(device)

Model(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 20, kernel_size=(5, 5), stride=(1, 1))
)